In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 352.00 KiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [2]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=1,
    lora_alpha=3,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'key', 'value', 'attention.output.dense'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 667,396 || all params: 125,316,104 || trainable%: 0.5326


In [3]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [4]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.361300,1.318448,0.725000
200,1.117800,0.606654,0.867188
300,0.445400,0.366432,0.879687
400,0.306000,0.352242,0.889062
500,0.322700,0.333524,0.896875
600,0.313500,0.331322,0.892188
700,0.315700,0.335089,0.893750
800,0.340500,0.318846,0.896875
900,0.310800,0.316771,0.901563
1000,0.301800,0.315560,0.901563


In [8]:
training_args = TrainingArguments(
    # Core training configs
    max_steps=3000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    lr_scheduler_type='linear',
    learning_rate=5e-5,

    # Logging, evaluation, and checkpointing
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    output_dir='/kaggle/working/saved_models',
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args, checkpoint='/kaggle/working/saved_models/checkpoint-2000')

Starting training...
Resuming from checkpoint: /kaggle/working/saved_models/checkpoint-2000


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3418: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint, OPTIMIZER_NAME), map_

Step,Training Loss,Validation Loss,Accuracy
2100,0.317100,0.311241,0.904687
2200,0.304800,0.309564,0.903125
2300,0.285500,0.310045,0.900000
2400,0.312700,0.311079,0.904687
2500,0.286200,0.308379,0.907813
2600,0.271100,0.308054,0.907813
2700,0.264500,0.311223,0.907813
2800,0.278300,0.307168,0.904687
2900,0.280700,0.308609,0.906250
3000,0.290300,0.308272,0.906250


In [11]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2500'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:59<00:00,  4.19it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2500.csv


In [10]:
import os
import pandas as pd

# Define paths
csv_path = "/kaggle/working/saved_models/checkpoint-3000/log_history.csv"
output_dir = "/kaggle/working/processed_data"
output_csv = os.path.join(output_dir, "processed_log_history.csv")

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the csv file
df = pd.read_csv(csv_path)

# Select desired columns and reorder the dataframe
desired_order = [
    "step",         
    "loss",         
    "eval_loss",   
    "accuracy",     
    "eval_accuracy",
    "learning_rate",
    "epoch"         
]
df = df[desired_order]

# Rename columns for uniformity
df.rename(columns={
    "step": "Step",
    "loss": "Train Loss",
    "eval_loss": "Test Loss",
    "accuracy": "Train Acc",
    "eval_accuracy": "Test Acc",
    "learning_rate": "Learning Rate",
    "epoch": "Epochs"
}, inplace=True)

# Adding loss spread and loss ratio columns
df["loss spread"] = df["Train Loss"] - df["Test Loss"]
df["loss ratio"] = df["Train Loss"] / df["Test Loss"]

# Adding acc spread and acc ratio columns
df["Acc spread"] = df["Test Acc"] - df["Train Acc"]
df["Acc ratio"] = df["Test Acc"] / df["Train Acc"]

# Export the DataFrame as a csv file
df.to_csv(output_csv, index=False)

print(f"Data exported to {output_csv}")

df

Data exported to /kaggle/working/processed_data/processed_log_history.csv


,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,loss spread,loss ratio,Acc spread,Acc ratio
0,100,1.3613,1.318448,0.384375,0.725000,0.000048,0.013405,0.042852,1.032502,0.340625,1.886179
1,200,1.1178,0.606654,0.715179,0.867188,0.000045,0.026810,0.511146,1.842565,0.152009,1.212547
2,300,0.4454,0.366432,0.868304,0.879687,0.000043,0.040214,0.078968,1.215505,0.011384,1.013111
3,400,0.3060,0.352242,0.899554,0.889062,0.000040,0.053619,-0.046242,0.868720,-0.010491,0.988337
4,500,0.3227,0.333524,0.895982,0.896875,0.000037,0.067024,-0.010824,0.967546,0.000893,1.000997
5,600,0.3135,0.331322,0.889732,0.892188,0.000035,0.080429,-0.017822,0.946209,0.002455,1.002760
6,700,0.3157,0.335089,0.897768,0.893750,0.000033,0.093834,-0.019389,0.942138,-0.004018,0.995525
7,800,0.3405,0.318846,0.885714,0.896875,0.000030,0.107239,0.021654,1.067915,0.011161,1.012601
8,900,0.3108,0.316771,0.895982,0.901563,0.000028,0.120643,-0.005971,0.981151,0.005580,1.006228
9,1000,0.3018,0.315560,0.902679,0.901563,0.000025,0.134048,-0.013760,0.956395,-0.001116,0.998764
